In [34]:
import tweepy
from tweepy import OAuthHandler, Stream
from tweepy.streaming import StreamListener
import json
import pandas as pd
import xlrd
import datetime
import pytz
import numpy as np

In [35]:
def is_rate_limit_error(e):
    return isinstance(e.message, list) \
        and e.message[0:] \
        and 'code' in e.message[0] \
        and e.message[0]['code'] == 88

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [50]:
## make list of files
pathin = ''
pathout= ''
filein = []

In [ ]:
for eachFile in filein:
    xls = pd.ExcelFile(pathin+eachFile+'.xls')
    df = xls.parse(0)
    df['Content'] = ""
    df['UTC'] = ""

    for i in range (0,len(df)):
        try:
            gettweet=api.get_status(df['GUID'][i])
            json_str = json.dumps(gettweet._json)
            gettweet=json_str.split(', "')
            tweet=gettweet[2].split(': "')[1][:(len(gettweet[2].split(': "')[1])-1)]
            gettweet.reverse()
            pref="created_at"
            try:
                check=gettweet[2].split(': "')[1][:(len(gettweet[2].split(': "')[1])-1)]
                if check=="photo" or check=="video" or check=="animated gif":
                    for j in range(0,len(gettweet)):
                        if gettweet[j].startswith(pref):
                            date=gettweet[j].split(': "')[1][:(len(gettweet[j].split(': "')[1])-1)]
                            break
                else:
                    date=check
            except IndexError:
                for j in range(0,len(gettweet)):
                    if gettweet[j].startswith(pref):
                        date=gettweet[j].split(': "')[1][:(len(gettweet[j].split(': "')[1])-1)]
                        break
            df['Content'][i] = tweet
            df['UTC'][i] = date
            print i, date
        except tweepy.TweepError, e:
            if not is_rate_limit_error(e):
                pass
            else:
                current=datetime.datetime.now()
                print current.hour,':',current.minute

    df.to_csv((pathout+eachFile+'.csv'), sep=',', encoding='utf-8')